## Setup
To use this notebook, run the following to install the dependencies. Note, the first time you use anything (library, function, etc), Julia compiles it. This means that writing `using CUDA` takes ~1m42s on my workstation the first time I run it. Be warned!

In [27]:
using Pkg; Pkg.add("CUDA"); Pkg.add("IntervalOptimisation"); Pkg.add("TreeView");

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


## First adventures

Julia is like python but also like C++. It also has tons of built in math, like numpy-style array operations are in by default

In [17]:
#An example of array arithmetic
v = [1.0, 2,3]
v+v

3-element Vector{Float64}:
 2.0
 4.0
 6.0

Note that Julia determined the type (Float64) from the literal's used (in this case the 1.0). It naturally has mathematical operations defined too, like vector addition. I can even do a dot product   

In [4]:
using LinearAlgebra
dot(v,v)

14.0

This isn't sexy enough. Write `v ` then `\cdot` and press tab to autocomplete (yes latex is the way these are defined), then put `v` and you get a dot product 

In [18]:
v ⋅ v

14.0

That's right, we have unicode, and a lot of it is plumbed into real mathematical definitions. Lets make a function, one that uses Julia's cool implicit multiply

In [62]:
f(x) = 2 + 5x

f (generic function with 1 method)

What is f?

In [64]:
code_lowered(f)

1-element Vector{Core.CodeInfo}:
 CodeInfo(
1 ─ %1 = 5 * x
│   %2 = 2 + %1
└──      return %2
)

Cool! I can see the code! Hmm, looks a little generic and slow. What if I put the type information in, do I see more code?

In [67]:
code_typed(f, (Float64,))

1-element Vector{Any}:
 CodeInfo(
1 ─ %1 = Base.mul_float(5.0, x)::Float64
│   %2 = Base.add_float(2.0, %1)::Float64
└──      return %2
) => Float64

Its compiling to some specialised operations for floats where its tracking the types, the values, and the arguments. Do we hit bare metal?

In [70]:
code_native(f, (Float64,))

	.text
	.file	"f"
	.section	.rodata.cst8,"aM",@progbits,8
	.p2align	3                               # -- Begin function julia_f_8971
.LCPI0_0:
	.quad	0x4014000000000000              # double 5
.LCPI0_1:
	.quad	0x4000000000000000              # double 2
	.text
	.globl	julia_f_8971
	.p2align	4, 0x90
	.type	julia_f_8971,@function
julia_f_8971:                           # @julia_f_8971
; ┌ @ /home/mjki2mb2/testJulia/test.ipynb:1 within `f`
	.cfi_startproc
# %bb.0:                                # %top
	movabsq	$.LCPI0_0, %rax
; │┌ @ promotion.jl:389 within `*` @ float.jl:385
	vmulsd	(%rax), %xmm0, %xmm0
	movabsq	$.LCPI0_1, %rax
; │└
; │┌ @ promotion.jl:388 within `+` @ float.jl:383
	vaddsd	(%rax), %xmm0, %xmm0
; │└
	retq
.Lfunc_end0:
	.size	julia_f_8971, .Lfunc_end0-julia_f_8971
	.cfi_endproc
; └
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


Awesome, assembly! All created whenever we use the function, but not before! So we always get compiled, fully optimised code. Wait, if we write abstract functions, that then get adapted to native code only when run, can we run on non-native devices?
# CUDA
Yes, just change the type

In [73]:
using CUDA
gpu_v = CuArray([1,2,3,4,5,6,7,8]) #This array now lives on the GPU

#Some generic function using the . to make .+ an elementwise addition
#f(x) = x + 3x ⋅ x * x .+ 12 

#f(gpu_v)

CuError: CUDA error: unknown error (code 999, ERROR_UNKNOWN)